In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

# to make a csr matrix (for fast count estimation) will be used libs below
from scipy import sparse
from pandas.api.types import CategoricalDtype

# to reduce amount of repeating code from notebook to another there is a lib of
#often used functions called
import sys
sys.path.insert(0, "C:/workspace/data-science/perpetual/second_investigation_wave")
from data_transformers import *

# Collect changes with opens and closes

In [11]:
changes_opens_closes_df = pd.read_csv("changes_opens_closes_df.csv")

In [12]:
changes_opens_closes_df.drop(columns=["Unnamed: 0"], inplace=True)
changes_opens_closes_df.sort_values(by=["timestamp", "tx_change_version", "open_id"], inplace=True)
changes_opens_closes_df.reset_index(inplace=True)
changes_opens_closes_df.drop(columns=["index"], inplace=True)
changes_opens_closes_df

,id,trader,amm,margin,position_notional,exchanged_position_size,fee,position_size_after,realized_pnl,unrealized_pnl_after,bad_debt,liquidation_penalty,spot_price,funding_payment,block_number,timestamp,amm_name,tx_change_version,time,open_id
0,0x0e824c5f4e27ba5f29b8791e25e76e732783b25bba44...,0x1a48776f436bcdaa16845a378666cf4ba131eb0f,0x8d22f1a9dce724d8c1b4c688d75f17a2fe2d32df,6816.007869,20448.023608,34.908392,20.448024,34.908392,0.000000e+00,0.000000,0.0,0.0,586.786590,0.000000,13508290,1607915765,ETH/USDC,6,2020-12-14 03:16:05,1.0
1,0xeebbca404104c295253184426d13273fcafcb7886773...,0x1a48776f436bcdaa16845a378666cf4ba131eb0f,0x8d22f1a9dce724d8c1b4c688d75f17a2fe2d32df,13632.015738,20448.023608,34.786745,20.448024,69.695137,0.000000e+00,71.380545,0.0,0.0,588.836755,0.000000,13508290,1607915765,ETH/USDC,13,2020-12-14 03:16:05,1.0
2,0x06d54c4ad4bfff493051ef548557f4fa0c76fb2dd678...,0x1a48776f436bcdaa16845a378666cf4ba131eb0f,0x0f346e19f01471c02485df1758cfd3d624e399b4,5552.445854,16657.337561,0.872710,16.657338,0.872710,0.000000e+00,0.000000,0.0,0.0,19120.287600,0.000000,13508290,1607915765,BTC/USDC,20,2020-12-14 03:16:05,1.0
3,0xe9d804889de5a27a356777a588d6c018e55fa1cae61c...,0x1a48776f436bcdaa16845a378666cf4ba131eb0f,0x0f346e19f01471c02485df1758cfd3d624e399b4,11114.600000,16686.462439,0.871187,16.686462,1.743896,0.000000e+00,58.249669,0.0,0.0,19187.208607,0.000000,13508954,1607919105,BTC/USDC,6,2020-12-14 04:11:45,1.0
4,0xea81e16520eae51395b4627365487daa2013df876ef3...,0x1a48776f436bcdaa16845a378666cf4ba131eb0f,0x0f346e19f01471c02485df1758cfd3d624e399b4,0.000000,33343.800000,-1.743896,33.343800,0.000000,-1.782200e-14,0.000000,0.0,0.0,19053.600000,0.000000,13509011,1607919390,BTC/USDC,6,2020-12-14 04:16:30,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962230,0xf8ec16b28757329c3ee24c7c15c485bf34e1a99e033b...,0x72dd2ce1b88f2cbcee606ec2ede14cf03563e08d,0x26789518695b56e16f14008c35dc1b281bd5fc0e,35072.825749,4000.388562,-10070.589592,4.000389,-126326.000688,0.000000e+00,82.878106,0.0,0.0,0.396956,0.000000,21634203,1649938115,ALPHA/USDC,6,2022-04-14 12:08:35,1.0
9962231,0x3a0be5f6d2b3f88505f19ed9e6370dbff74b5619baae...,0xd22084c2444943db47c6eff3084b087f5e8f77de,0x26789518695b56e16f14008c35dc1b281bd5fc0e,3583.867495,2621.399982,-6606.807256,2.621400,-10044.712580,0.000000e+00,3.325167,0.0,0.0,0.396590,0.000000,21634203,1649938115,ALPHA/USDC,13,2022-04-14 12:08:35,1076.0
9962232,0x6c23451dcaa2fc33eceaa6b90800ddd9f065ff309e5e...,0xc701072cd7396622fac184aa778d549a34b1545c,0xd41025350582674144102b74b8248550580bb869,0.000000,2130.790709,0.112192,2.130791,0.000000,1.220929e+01,0.000000,0.0,0.0,19001.851415,0.265515,21634205,1649938125,YFI/USDC,6,2022-04-14 12:08:45,3963.0
9962233,0x443acaa5b44dd26efd6605cf13425c3f1340eb76508c...,0x7a4ee543471f91ed1f8b33a476af4adbd15da4db,0xf559668108ff57745d5e3077b0a7dd92ffc6300c,19036.824763,2511.211682,746.050228,2.511212,-7688.687621,-3.449239e+00,-35.547370,0.0,0.0,3.368384,0.000000,21634218,1649938190,SUSHI/USDC,4,2022-04-14 12:09:50,3.0


# Make a map of trader's presence on the pool

In [13]:
amm_trader_list = []

for amm in changes_opens_closes_df['amm_name'].unique():
    amm_story = changes_opens_closes_df[changes_opens_closes_df['amm_name'] == amm]
    amm_trader_df = pd.DataFrame(columns=['amm', 'trader'])
    amm_trader_df['trader'] = amm_story['trader'].unique()
    amm_trader_df['amm'] = [amm] * len(amm_story['trader'].unique())
    amm_trader_list.append(amm_trader_df)
    
traders_amm_df = pd.concat(amm_trader_list)
traders_amm_df["count"] = [1] * len(traders_amm_df)
traders_amm_df

,amm,trader,count
0,ETH/USDC,0x1a48776f436bcdaa16845a378666cf4ba131eb0f,1
1,ETH/USDC,0x9602686bf53a17baed60c48ba34ed4219a532381,1
2,ETH/USDC,0x5bea7caa02142ba59885466757e1112819731f09,1
3,ETH/USDC,0x3f84e391ea8dc12946e17d1d85cdf0b35d4fe650,1
4,ETH/USDC,0x43bd90e4cc93d6e40580507102cc7b1bc8a25284,1
...,...,...,...
563,FTT/USDC,0xb39885f302643a5f96f743c3621d628dc3f72ce6,1
564,FTT/USDC,0xa84daa76815f6d2cf99e406bd693e8588bc36dbe,1
565,FTT/USDC,0xfe8b307829c386bc056e43fa2f4a9edc7a7a62ca,1
566,FTT/USDC,0xbd9417537b91eedd4c0ccda78052762380350f0b,1


In [14]:
users = traders_amm_df["trader"].unique()
amms = traders_amm_df["amm"].unique()
shape = (len(users), len(amms))

user_cat = CategoricalDtype(categories=sorted(users), ordered=True)
amms_cat = CategoricalDtype(categories=sorted(amms), ordered=True)

user_index = traders_amm_df["trader"].astype(user_cat).cat.codes
amm_index = traders_amm_df["amm"].astype(amms_cat).cat.codes

# Conversion via COO matrix
coo = sparse.coo_matrix((traders_amm_df["count"], (user_index, amm_index)), shape=shape)
csr = coo.tocsr()

In [15]:
stats_df = pd.DataFrame.sparse.from_spmatrix(csr, columns=amms_cat.categories)
stats_df.index = user_cat.categories
stats_df['occurencies_sum'] = stats_df.sum(axis=1)
stats_df

,AAVE/USDC,ALPHA/USDC,BTC/USDC,COMP/USDC,CREAM/USDC,CRV/USDC,DOT/USDC,ETH/USDC,FTT/USDC,GRT/USDC,LINK/USDC,MKR/USDC,PERP/USDC,REN/USDC,SNX/USDC,SUSHI/USDC,UNI/USDC,YFI/USDC,unknown,occurencies_sum
0x000000003b0371b33aba8b6b58154a98d54695ad,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
0x000000064730640b7d670408d74280924883064f,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
0x0000000813b34008a225de08a6a61835508c71f9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0x000000ea89990a17ec07a35ac2bbb02214c50152,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,17
0x0000ce08fa224696a819877070bf378e8b131acf,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0xffe694afb3b54ee0add6ded4cc0f9917777f13da,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
0xfffb40347816873ca038756cc87bc610144166b2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
0xfffdd2572f57b6457c8c4b0320e8f1f1ec4812bd,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
0xfffe45d946f1f715dc0252df3db22a9edfaf5738,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


It is required to count traders conform count of pools where their activity was registered

*   Presence in single pool = 8288 traders;
*   Presence in 2 pools = 694 traders;
*   Presence in 3 pools = 291 traders;
*   Presence in 4 pools = 190 traders;
*   Presence in 5 pools = 124 traders;
*   Presence in 6 pools = 101 traders;
*   Presence in 7 pools = 68 traders;
*   Presence in 8 pools = 77 traders;
*   Presence in 9 pools = 63 traders;
*   Presence in 10 pools = 50 traders;
*   Presence in 11 pools = 44 traders;
*   Presence in 12 pools = 40 traders;
*   Presence in 13 pools = 38 traders;
*   Presence in 14 pools = 26 traders;
*   Presence in 15 pools = 14 traders;
*   Presence in 16 pools = 15 traders;
*   Presence in 17 pools = 16 traders;
*   Presence in 18 pools = 60 traders;

In [16]:
len(stats_df[stats_df["occurencies_sum"] == 18])

60

# Calculate frequency of transactions for each trader on each pool separately

In [17]:
stats_df

,AAVE/USDC,ALPHA/USDC,BTC/USDC,COMP/USDC,CREAM/USDC,CRV/USDC,DOT/USDC,ETH/USDC,FTT/USDC,GRT/USDC,LINK/USDC,MKR/USDC,PERP/USDC,REN/USDC,SNX/USDC,SUSHI/USDC,UNI/USDC,YFI/USDC,unknown,occurencies_sum
0x000000003b0371b33aba8b6b58154a98d54695ad,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
0x000000064730640b7d670408d74280924883064f,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
0x0000000813b34008a225de08a6a61835508c71f9,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0x000000ea89990a17ec07a35ac2bbb02214c50152,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,17
0x0000ce08fa224696a819877070bf378e8b131acf,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0xffe694afb3b54ee0add6ded4cc0f9917777f13da,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
0xfffb40347816873ca038756cc87bc610144166b2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
0xfffdd2572f57b6457c8c4b0320e8f1f1ec4812bd,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
0xfffe45d946f1f715dc0252df3db22a9edfaf5738,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1


In [18]:
frequencies_list = []

for trader in stats_df.index:
    trader_changes_opens_closes_df = changes_opens_closes_df[changes_opens_closes_df["trader"] == trader]
    activity_time = trader_changes_opens_closes_df["timestamp"].max() - trader_changes_opens_closes_df["timestamp"].min()
    tx_count = len(trader_changes_opens_closes_df)
    if (activity_time == 0):
        frequencies_list.append(0)
    else:
        frequencies_list.append(tx_count / 
                                activity_time)
    
stats_df["tx_frequency"] = frequencies_list  

In [20]:
stats_df.to_csv("trader_activity_stats_df.csv")